In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini chat - Multi-turn

## Set environment

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [24]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

### Import libraries

* [Package vertexai (1.75.0)](https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai?_gl=1*1nptmcf*_ga*MzMxNjc2MjguMTczNDM3MTI2Mg..*_ga_WH2QY8WWF5*MTczNTEyOTIyNy4yNS4xLjE3MzUxMjk4NTQuNTkuMC4w)


In [25]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import grounding


### Initalize Vertex AI
Use system instructions when using multi-turn to keep consistency of communication.


In [26]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

system_instructions = [
 "당신은 가전제품을 구매하도록 도와주는 어시스턴트 입니다.",
 "질문 내용에 상세하게 답해주세요. "
]

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Helper function

In [27]:
def interactive_chat(chat: ChatSession, question: str) -> str:
    """
    Generate a response from the model.

    Args:
      chat: ChatSession
        query string to generate response
      question: str
        whether to stream the response

    Returns:
      str: The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Google Search grounding.
    tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        question,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = [tool],
        stream=False
    )

    return responses.text

#------------------------------------------------------------------------------------------------------------------------

def get_chat_history(chat):

  """
  Generate a response from the model.

  Args:
    chat: ChatSession
      query string to generate response

  Returns:
    None

  """

  history = " ".join([content.text for content in chat.history])

  print(f"---------------- Chat history ----------------\n{history} ")



### Execution

In [28]:
chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  # chat_history = get_chat_history(chat)

  prompt = f"""
  당신은 가전제품을 구매하도록 도와 주는 AI Assistant 입니다.
  아래 질문에 대해서 친절하게 10줄 이내로 답해주세요.
  당신은 사용자가 적합한 제품을 살수 있도록 도와 줘야 합니다.

  질문 : {query}
  """

  response = interactive_chat(chat, prompt)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")


사용자: 안녕하세요.


AI Agent : 안녕하세요! 가전제품 구매를 도와드리겠습니다.  어떤 제품을 찾으시는지, 예산은 어느정도인지,  어떤 기능이 중요한지 알려주시면  더욱 정확하게 안내해 드릴 수 있습니다.  오프라인 매장(LG베스트샵, 삼성디지털프라자, 하이마트 등)과 온라인(네이버쇼핑, G마켓, 쿠팡 등) 모두  가격 비교 후 구매하시는 것을 추천드립니다. 특히, 제조사 직영점이나 이벤트 기간을 활용하면 더욱 저렴하게 구매할 수 있습니다.  모델별 스펙 비교도 꼼꼼히 하시고,  사은품이나 추가 할인 혜택도 확인해 보세요.  궁금한 점이 있으면 언제든지 질문해주세요!


------------------------------------ 
사용자: 냉장고를 사려고 하는데 어떤 제품이 좋을까요 ? 저는 4인가구 입니다. 


AI Agent : 4인 가구이시라면 600~800L 용량의 냉장고를 추천합니다.  LG 디오스 오브제컬렉션이나 삼성 비스포크 4도어 냉장고가 인기가 많습니다.  예산과 디자인 선호도에 따라  모델을 선택하시면 됩니다.  필요한 기능 (정수기, 얼음정수기, 냉장/냉동 용량 비율)도 고려해보세요.  온라인 판매 사이트에서 가격비교 후 구매하시는 것을 추천하며,  에너지 효율 등급(1등급이 가장 좋음)도 확인하세요.  매장에 방문하여 직접 제품을 확인하는 것도 좋은 방법입니다.  100만원대부터 200만원대 이상의 다양한 가격대의 제품들이 있으니 예산을 고려하여 선택하세요.


------------------------------------ 
사용자: 삼성전자 제품은 없나요 ?


...

------------------------------------ 
사용자: 최근 삼성전자 제품은 언제 출시되었나요 ?


AI Agent : 삼성전자의 최근 제품 출시 정보는 다양합니다.  가장 최근 자료를 바탕으로 말씀드리면, 2024년 3월에는  AI 기능을 강화한 '비스포크 냉장고' 신제품이 출시되었고,  2024년 7월에는 갤럭시 Z 폴드6와 Z플립6가 출시되었습니다.  또한,  2024년 9월에는 갤럭시 Z 스페셜 에디션과 갤럭시 S24 FE 출시 계획이 발표되었습니다.  이 외에도  연도별, 제품군별로 다양한 제품들이 출시되었으니 삼성전자 공식 웹사이트를 참고하시면 더욱 자세한 정보를 얻으실 수 있습니다.  제공된 정보는 2024년 12월 25일 현재까지의 정보입니다.


------------------------------------ 
사용자: 냉장고는 언제 출시 되었죠 ?


AI Agent : 제공된 자료에 따르면, 삼성전자는 2024년 3월 14일에 AI 기술을 강화한 '비스포크 냉장고' 신제품을 출시했습니다.  또한, 2024년 8월 28일에는 AI 기능을 강화한 새로운 '양문형 냉장고'를 출시했습니다.  이 두 제품 모두  최근 출시된 삼성전자 냉장고 제품입니다.  더욱 자세한 정보는 삼성전자 공식 웹사이트에서 확인하실 수 있습니다.  (이 답변은 2024년 12월 25일 현재 정보를 바탕으로 작성되었습니다.)


------------------------------------ 
사용자: 종료
